In [1]:
%load_ext dotenv
%dotenv 
import os
import sys
sys.path.append(os.getenv('SRC_DIR'))
from logger import get_logger
_logs = get_logger(__name__)

In [2]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import os
from glob import glob

c:\Users\bouwa\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:15: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


In [3]:
ft_dir = os.getenv("FEATURES_DATA")
ft_glob = glob(ft_dir+'/*.parquet')
df = dd.read_parquet(ft_glob).compute().reset_index()

ValueError: empty urlpath sequence

In [5]:
ft_dir = os.getenv("FEATURES_DATA")
print(f"FEATURES_DATA path: {ft_dir}")

FEATURES_DATA path: ../../05_src/data/features/stock_features.parquet


In [6]:
ft_dir = os.getenv("FEATURES_DATA")  # Ensure this is the full path to a single file

if not ft_dir or not os.path.exists(ft_dir):
    raise ValueError(f"File not found: {ft_dir}")

df = dd.read_parquet(ft_dir).compute().reset_index()

ValueError: File not found: ../../05_src/data/features/stock_features.parquet

In [7]:
import os

ft_dir = os.getenv("FEATURES_DATA")  # Get the environment variable
print(f"Resolved path: {os.path.abspath(ft_dir)}")  # Convert to absolute path

# Check if the file exists
if os.path.exists(ft_dir):
    print("File exists ✅")
else:
    print("File NOT found ❌")


Resolved path: c:\Users\bouwa\Desktop\Anis Bwz\Learning\DSI\production\05_src\data\features\stock_features.parquet
File NOT found ❌


In [8]:
!ls -l ../../05_src/data/features/


'ls' is not recognized as an internal or external command,
operable program or batch file.


In [11]:
ft_dir = os.getenv("FEATURES_DATA")

if not ft_dir or not os.path.exists(ft_dir):
    raise ValueError(f"Directory not found: {os.path.abspath(ft_dir)}")

df = dd.read_parquet(ft_dir, engine="pyarrow").compute().reset_index()

ValueError: Directory not found: c:\Users\bouwa\Desktop\Anis Bwz\Learning\DSI\production\05_src\data\features\stock_features.parquet

# Sampling in Python

+ There are different packages that allow sampling.
+ A practical approach is to use pandas/Dask sampling methods.

## Random Sampling

+ Sample n rows from a dataframe with [`df.sample()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html).

```
DataFrame.sample(
    n=None, frac=None, replace=False, weights=None, 
    random_state=None, axis=None, ignore_index=False
    )
```

In [1]:
df.sample(n = 5)

NameError: name 'df' is not defined

In [14]:
# select_tickers = df['ticker'].unique().tolist()[0:9]
select_tickers = ['OXY', 'TXN', 'PCG', 'CBRE', 'ROK', 'AMT', 'D', 'YUM', 'PWR']
select_tickers

['OXY', 'TXN', 'PCG', 'CBRE', 'ROK', 'AMT', 'D', 'YUM', 'PWR']

In [15]:
idx = df['Ticker'].isin(select_tickers)
small_df = df[idx]
simple_sample_dt = small_df.sample(frac = 0.1)
simple_sample_dt.shape, small_df.shape

NameError: name 'df' is not defined

Look at the distribution of tickers.

In [ ]:
small_df['ticker'].value_counts().plot(kind='bar')

In [ ]:
simple_sample_dt['ticker'].value_counts().plot(kind='bar')

## Stratified Sampling

+ Use `groupby()` and `.sample()` for stratified sampling.

In [ ]:
strat_sample_dt = small_df.groupby(['ticker']).sample(frac = 0.1)
strat_sample_dt['ticker'].value_counts().plot(kind='bar')

# Sampling in Dask

+ Stratified sampling in `dask` can be achieved with `groupby().apply()` and a lambda function.

In [ ]:
dd_dt = dd.read_parquet(ft_glob)
strat_sample_dd_dt = (dd_dt
                      .groupby('ticker', group_keys=False)
                      .apply(lambda x: x.sample(frac = 0.1))
                      .compute()
                      .reset_index())
strat_sample_dd_dt[strat_sample_dd_dt['ticker'].isin(select_tickers)]['ticker'].value_counts().plot(kind='bar')